<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/childpark_congestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle

Uninstalling kaggle-1.5.6:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.6/dist-packages/kaggle-1.5.6.dist-info/*
    /usr/local/lib/python3.6/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 4.6MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 2.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=94aa3bafda0d6a36eb4d716c5118104bc7176823a69652a573c7c072f01f9be0
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 19 13:55 kaggle.json


In [3]:
! kaggle competitions download -c childpark

  0% 0.00/41.2k [00:00<?, ?B/s]
100% 41.2k/41.2k [00:00<00:00, 28.8MB/s]


In [4]:
!unzip childpark.zip

Archive:  childpark.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train_classi.csv        


In [248]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

In [249]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [250]:
xy = pd.read_csv('train_classi.csv', header=None)
print(xy)

xy=pd.DataFrame.dropna(xy, axis=0, how='any', thresh=None, subset=None, inplace=False)
x_data = xy.loc[1:,1:8]
y_data = xy.loc[1:, 9]



date = xy.loc[1:,0]
A = date.str.extract(r'(\d+)[:]', expand=True)
print(A)
x_data["date1"] = A
x_data["date2"] = A

x_data = x_data.apply(pd.to_numeric)
y_data = y_data.apply(pd.to_numeric)

x_data = np.array(x_data)
y_data = np.array(y_data)



minMaxScaler = MinMaxScaler()
print(minMaxScaler.fit(x_data))
x_data = minMaxScaler.transform(x_data)

print(x_data[5:15])
print(y_data[5:15])
print(x_data.shape)
print(y_data.shape)


x_train=torch.FloatTensor(x_data)
y_train=torch.LongTensor(y_data)


train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=8,
                                          shuffle=True,
                                          drop_last=True)

                     0             1     2  ...        7       8           9
0                 Date  Temperatures  Rain  ...  Holiday  Riding  Congestion
1      2019-01-01 5:00          -7.6     0  ...        1     107           0
2      2019-01-01 6:00          -7.9     0  ...        1     209           0
3      2019-01-01 7:00          -7.7     0  ...        1     150           0
4      2019-01-01 8:00          -7.7     0  ...        1     180           0
...                ...           ...   ...  ...      ...     ...         ...
2995  2019-06-30 19:00          24.3     0  ...        1     515           0
2996  2019-06-30 20:00          23.9     0  ...        1     471           0
2997  2019-06-30 21:00          23.4     0  ...        1     429           0
2998  2019-06-30 22:00            23     0  ...        1     406           0
2999  2019-06-30 23:00          22.7     0  ...        1     190           0

[3000 rows x 10 columns]
       0
1      5
2      6
3      7
4      8
5    

In [251]:
nb_class=3
nb_data=len(y_train)

In [293]:
layer1 = torch.nn.Linear(10, 16)
layer2 = torch.nn.Linear(16,16)
layer3 = torch.nn.Linear(16, nb_class)
relu = torch.nn.ReLU()
torch.nn.init.kaiming_normal_(layer1.weight)
torch.nn.init.kaiming_normal_(layer2.weight)
torch.nn.init.kaiming_normal_(layer3.weight)

model = torch.nn.Sequential(layer1,relu, 
                            layer2,relu, 
                            layer3)

In [294]:
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) 

In [301]:
total_batch = len(data_loader)
model_h = []
error_h = []
for epoch in range(1, 1+150000):
  avg_cost = 0
  optimizer.zero_grad()
  hypothesis = model(x_train)
        
  cost = loss(hypothesis, y_train)
  cost.backward()
  optimizer.step()
  avg_cost += cost
  avg_cost /= total_batch
  model_h.append(model)
  error_h.append(avg_cost)

  if epoch % 50 == 1 :
        print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch: 100101 cost = 0.000394089
Epoch: 100151 cost = 0.000379865
Epoch: 100201 cost = 0.000387501
Epoch: 100251 cost = 0.000380253
Epoch: 100301 cost = 0.000368174
Epoch: 100351 cost = 0.000393298
Epoch: 100401 cost = 0.000368069
Epoch: 100451 cost = 0.000371949
Epoch: 100501 cost = 0.000374513
Epoch: 100551 cost = 0.000368545
Epoch: 100601 cost = 0.000387534
Epoch: 100651 cost = 0.000368204
Epoch: 100701 cost = 0.000373571
Epoch: 100751 cost = 0.000379106
Epoch: 100801 cost = 0.000381820
Epoch: 100851 cost = 0.000367948
Epoch: 100901 cost = 0.000365184
Epoch: 100951 cost = 0.000366979
Epoch: 101001 cost = 0.000370340
Epoch: 101051 cost = 0.000364597
Epoch: 101101 cost = 0.000365990
Epoch: 101151 cost = 0.000377828
Epoch: 101201 cost = 0.000392427
Epoch: 101251 cost = 0.000367570
Epoch: 101301 cost = 0.000373226
Epoch: 101351 cost = 0.000367904
Epoch: 101401 cost = 0.000367713
Epoch: 101451 cost = 0.000370190
Epoch: 101501 cost = 0.000367026
Epoch: 

In [302]:
best_model = model_h[np.argmin(error_h)]

In [303]:

test = pd.read_csv('test.csv', header=None)
test=pd.DataFrame.dropna(test, axis=0, how='any', thresh=None, subset=None, inplace=False)
print(test)
test_data = test.loc[1:,1:8]


date_t = test.loc[1:,0]

B = date_t.str.extract(r'(\d+)[:]', expand=True)
print(B)
test_data["date1"] = B
test_data["date2"] = B

test_data = test_data.apply(pd.to_numeric)

test_data=np.array(test_data)

minmaxScaler = MinMaxScaler()
print(minmaxScaler.fit(test_data))
test_data = minmaxScaler.transform(test_data)

print(test_data[:5])
test_data=torch.FloatTensor(test_data)
print(test_data[0])

                    0             1     2     3  ...     5      6        7       8
0                Date  Temperatures  Rain  Wind  ...  Snow  Cloud  Holiday  Riding
1     2019-05-01 5:00          11.5     0   0.7  ...     0      3        0     128
2     2019-05-01 6:00          11.3     0   0.3  ...     0      7        0     199
3     2019-05-01 7:00          11.5     0   0.8  ...     0      3        0     343
4     2019-05-01 8:00          12.9     0     1  ...     0      0        0     678
..                ...           ...   ...   ...  ...   ...    ...      ...     ...
616  2019-05-31 20:00          18.2     0     4  ...     0      0        0     794
617  2019-05-31 21:00          17.3     0   2.4  ...     0      0        0     816
618  2019-05-31 22:00          16.8     0   2.6  ...     0      0        0     882
619  2019-05-31 23:00          16.5     0   2.2  ...     0      0        0     623
620   2019-06-01 0:00          16.1     0   1.3  ...     0      0        0     147

[62

In [304]:
with torch.no_grad():
    #test_data = test_data.to(device)
    pred = best_model(test_data)
    predict=torch.argmax(pred,dim=1)

    print(predict.shape)

torch.Size([620])


In [305]:
submit = pd.read_csv('submit.csv')

predict=predict.cpu().numpy().reshape(-1,1)

id=np.array([i for i in range(len(predict))]).reshape(-1,1)
result=np.hstack([id,predict])

submit=pd.DataFrame(result,columns=["id","result"])
submit.to_csv("submit.csv",index=False,header=True)

In [306]:
!kaggle competitions submit -c childpark -f submit.csv -m "14010974_이기택"

100% 3.54k/3.54k [00:02<00:00, 1.42kB/s]
Successfully submitted to Congestion in Subway stations